2 models for Intents with & without Over sampling(SMOTE)- Taken >50 count

In [1]:
import tensorflow as tf
import tensorflow.keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd

df=pd.read_csv('df_cleaned.csv')
df.head()

,cspro_primary_topic,transcript
0,DB - Payments,agent hello craig tom hey thanks hey something...
1,HW - Annual Enrollment,agent good morning thank calling eighteen bene...
2,HW - Annual Enrollment,agent good morning thank calling teensy benefi...
3,CM - Access,agent thank calling chancy dennis center name ...
4,HW - Annual Enrollment,agent moment please call ahead customer hey ma...


In [3]:
df=df.drop_duplicates(subset='transcript')
df.shape

(72559, 2)

In [4]:
print(df.cspro_primary_topic.nunique())

print((df.cspro_primary_topic.value_counts()>50).sum())

146
51


In [5]:
df1 = df[df.groupby('cspro_primary_topic')['cspro_primary_topic'].transform('size') > 50]
df1.shape

(71473, 2)

In [6]:
df1.cspro_primary_topic.value_counts()

HW - Annual Enrollment            17109
CM - Access                        8860
HW - COBRA                         8260
CM - External Referral             5300
CM - Internal Warm Transfer        3712
DB - Payments                      3219
HRFS Call                          3210
COBRA/Direct Billing               2961
HW - Life Insurance                2210
CM - Workflow                      1729
Referral                           1652
CM - LOA                           1604
Payroll - Ongoing Processing       1126
Qualified Status Change             962
Time and Labor - PTO                915
DC - Regular Withdrawals            902
Payroll - Tax                       685
DC - Rollover In                    653
Data Issues                         646
YSA - Web                           501
YSA - Active HRA                    486
PAY - Paycheck Assistance           380
YSA - Commuter                      302
CM - Case Management                282
DB - Lump Sum Window                279


In [7]:


vocab_size = 50000
max_length = 300
trunc_type = 'post'
padding_type = 'post'

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(df1['transcript'])
word_index = tokenizer.word_index

In [8]:
dict(list(word_index.items())[0:10])

{'agent': 1,
 'customer': 2,
 'okay': 3,
 'redacted': 4,
 'know': 5,
 'thank': 6,
 'call': 7,
 'yes': 8,
 'one': 9,
 'get': 10}

In [9]:
print(len(word_index))

44154


In [10]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [11]:
df1['transcript'] = tokenizer.texts_to_sequences(df1['transcript'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
print(df['transcript'][0])
print(df1['transcript'][0])

agent hello craig tom hey thanks hey something give call hey bye bye okay greatly okay okay okay thanks bye compression thank bye bye give quick quick schedule going hey okay thanks point okay thanks bye hello hey quickly thank thank bye call thank bye hey good hey okay hey bye thanks think calling call thank bye bye bye okay thank goodbye okay company working cleaning bye bye bye okay hey thanks okay thank bye thanks look customer please agent call thanks okay okay leaf okay bye hey thanks bye please conclude know bye okay bye okay speaking thank okay talk okay thanks okay call today okay thank bye okay thank bye okay okay thanks bye okay okay okay bye bye salmon time thank commander greg pickering one clock bye norman okay bye thanks bye claim calling customer okay okay
[1, 53, 2142, 777, 90, 147, 90, 99, 28, 7, 90, 16, 16, 3, 2096, 3, 3, 3, 147, 16, 14056, 6, 16, 16, 28, 427, 427, 840, 50, 90, 3, 147, 306, 3, 147, 16, 53, 90, 1066, 6, 6, 16, 7, 6, 16, 90, 49, 90, 3, 90, 16, 147, 73,

In [13]:
X= pad_sequences(df1['transcript'], maxlen=max_length)

In [14]:
X[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [15]:
print(len(df1['transcript'][0]))
print(len(X[0]))

print(len(df1['transcript'][1]))
print(len(X[1]))

print(len(df1['transcript'][10]))
print(len(X[10]))

140
300
329
300
342
300


In [16]:
y=df1['cspro_primary_topic'].copy()

In [17]:
from sklearn.preprocessing import LabelEncoder

y = LabelEncoder().fit_transform(y.values)

In [18]:
y.shape

(71473,)

In [23]:
#Train test split of data without sampling

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(57178, 300) (57178,)
(14295, 300) (14295,)


In [49]:
!pip install imblearn --index-url https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple

Looking in indexes: https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/simple
  Using cached https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/packages/packages/81/a7/4179e6ebfd654bd0eac0b9c06125b8b4c96a9d0a8ff9e9507eb2a26d2d7e/imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/packages/packages/80/98/dc784205a7e3034e84d41ac4781660c67ad6327f2f5a80c568df31673d1c/imbalanced_learn-0.8.0-py3-none-any.whl (206 kB)
  Using cached https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/packages/packages/d3/eb/d0e658465c029feb7083139d9ead51000742e88b1fb7f1504e19e1b4ce6e/scikit_learn-0.24.2-cp36-cp36m-manylinux2010_x86_64.whl (22.2 MB)
  Using cached https://artifactory.alight.com/artifactory/api/pypi/python-pypi-remote/packages/packages/c6/e8/c216b9b60cbba4642d3ca1bae7a53daa0c24426f662e0e3ce3dc7f6caeaa/threadpoolctl-2.2.0-py3-none-any.whl (12 kB)
  Attempting un

In [19]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
Xs, ys = sm.fit_resample(X, y)

In [28]:
import numpy as np

unique1, counts1 = np.unique(ys, return_counts=True)

np.asarray((unique1, counts1)).T

array([[    0, 17109],
       [    1, 17109],
       [    2, 17109],
       [    3, 17109],
       [    4, 17109],
       [    5, 17109],
       [    6, 17109],
       [    7, 17109],
       [    8, 17109],
       [    9, 17109],
       [   10, 17109],
       [   11, 17109],
       [   12, 17109],
       [   13, 17109],
       [   14, 17109],
       [   15, 17109],
       [   16, 17109],
       [   17, 17109],
       [   18, 17109],
       [   19, 17109],
       [   20, 17109],
       [   21, 17109],
       [   22, 17109],
       [   23, 17109],
       [   24, 17109],
       [   25, 17109],
       [   26, 17109],
       [   27, 17109],
       [   28, 17109],
       [   29, 17109],
       [   30, 17109],
       [   31, 17109],
       [   32, 17109],
       [   33, 17109],
       [   34, 17109],
       [   35, 17109],
       [   36, 17109],
       [   37, 17109],
       [   38, 17109],
       [   39, 17109],
       [   40, 17109],
       [   41, 17109],
       [   42, 17109],
       [   

In [29]:
#Train test split of over-sampled data

X_train1, X_test1, y_train1, y_test1 = train_test_split(Xs, ys, test_size=0.2, random_state=42)

In [ ]:
#Model with over sampled data


from tensorflow.keras import Sequential
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping



embedding_dim=100
model1 = Sequential()
model1.add(Embedding(vocab_size,embedding_dim,input_length = X.shape[1]))
model1.add(SpatialDropout1D(0.2))
model1.add(LSTM(100,dropout = 0.2, recurrent_dropout = 0.2))
model1.add(Dense(98,activation = "softmax"))
model1.compile(loss = "sparse_categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])

EPOCHS = 50
batch_size = 64

history1 = model1.fit(X_train1,y_train1, epochs = EPOCHS,batch_size = batch_size,validation_split=0.1)

Train on 628242 samples, validate on 69805 samples
Epoch 1/50
628242/628242 [==============================] - 3637s 6ms/sample - loss: 3.2964 - accuracy: 0.1824 - val_loss: 2.7850 - val_accuracy: 0.3083
Epoch 2/50
628242/628242 [==============================] - 3659s 6ms/sample - loss: 2.7456 - accuracy: 0.3175 - val_loss: 2.4973 - val_accuracy: 0.3794
Epoch 3/50
628242/628242 [==============================] - 3638s 6ms/sample - loss: 2.5725 - accuracy: 0.3575 - val_loss: 2.3500 - val_accuracy: 0.4136
Epoch 4/50
628242/628242 [==============================] - 3648s 6ms/sample - loss: 2.4671 - accuracy: 0.3838 - val_loss: 2.2770 - val_accuracy: 0.4316
Epoch 5/50
628242/628242 [==============================] - 3811s 6ms/sample - loss: 2.3981 - accuracy: 0.3998 - val_loss: 2.2124 - val_accuracy: 0.4459
Epoch 6/50
628242/628242 [==============================] - 3807s 6ms/sample - loss: 2.3468 - accuracy: 0.4107 - val_loss: 2.1809 - val_accuracy: 0.4544
Epoch 7/50
628242/628242 [=====

In [47]:
#Model without over sampled data

from tensorflow.keras import Sequential
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping

embedding_dim=100
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100,dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(98,activation = "softmax"))
model.compile(loss = "sparse_categorical_crossentropy",optimizer = "adam", metrics = ["accuracy"])

EPOCHS = 50
batch_size = 64

history = model.fit(X_train,y_train, epochs = EPOCHS,batch_size = batch_size,validation_split=0.1)

Train on 51460 samples, validate on 5718 samples
Epoch 1/50
51460/51460 [==============================] - 314s 6ms/sample - loss: 2.7696 - accuracy: 0.2551 - val_loss: 2.5387 - val_accuracy: 0.3171
Epoch 2/50
51460/51460 [==============================] - 311s 6ms/sample - loss: 2.2964 - accuracy: 0.3822 - val_loss: 2.0497 - val_accuracy: 0.4598
Epoch 3/50
51460/51460 [==============================] - 311s 6ms/sample - loss: 1.9432 - accuracy: 0.4772 - val_loss: 1.8600 - val_accuracy: 0.4951
Epoch 4/50
51460/51460 [==============================] - 311s 6ms/sample - loss: 1.7220 - accuracy: 0.5267 - val_loss: 1.6952 - val_accuracy: 0.5421
Epoch 5/50
51460/51460 [==============================] - 311s 6ms/sample - loss: 1.5505 - accuracy: 0.5712 - val_loss: 1.6091 - val_accuracy: 0.5624
Epoch 6/50
51460/51460 [==============================] - 311s 6ms/sample - loss: 1.4095 - accuracy: 0.6038 - val_loss: 1.5741 - val_accuracy: 0.5654
Epoch 7/50
51460/51460 [===========================

KeyboardInterrupt: 